Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.527604
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.710562
Minibatch accuracy: 37.5%
Validation accuracy: 39.4%
Minibatch loss at step 100: 0.760359
Minibatch accuracy: 75.0%
Validation accuracy: 68.0%
Minibatch loss at step 150: 0.655316
Minibatch accuracy: 75.0%
Validation accuracy: 77.3%
Minibatch loss at step 200: 0.417300
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 250: 1.559070
Minibatch accuracy: 50.0%
Validation accuracy: 79.2%
Minibatch loss at step 300: 0.673191
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.520537
Minibatch accuracy: 87.5%
Validation accuracy: 79.1%
Minibatch loss at step 400: 0.475150
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 450: 0.338664
Minibatch accuracy: 93.8%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.470691
Minibatch accuracy: 87.5%
Validation accuracy: 81.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [59]:
from math import ceil

batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

def calc_HW(in_size, conv_stride, pool_stride):
    layer1_conv_HW = ceil(in_size / conv_stride)
    layer1_pool_HW = ceil(layer1_conv_HW / pool_stride)
    layer2_conv_HW = ceil(layer1_pool_HW / conv_stride)
    layer2_pool_HW = ceil(layer2_conv_HW / pool_stride)
    return int(layer2_pool_HW)

def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=init_weight_stddev(patch_size*patch_size*num_channels)))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=init_weight_stddev(patch_size*patch_size*depth)))
  layer2_biases = tf.Variable(tf.zeros([depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth, num_hidden], stddev=init_weight_stddev(calc_HW(image_size, 2, 2) * calc_HW(image_size, 2, 2) * depth)))
  layer3_biases = tf.Variable(tf.zeros([num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=init_weight_stddev(num_hidden)))
  layer4_biases = tf.Variable(tf.zeros([num_labels]))

  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    relu = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(relu, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [60]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.411443
Minibatch accuracy: 0.0%
Validation accuracy: 15.0%
Minibatch loss at step 50: 1.182375
Minibatch accuracy: 68.8%
Validation accuracy: 51.2%
Minibatch loss at step 100: 0.617075
Minibatch accuracy: 75.0%
Validation accuracy: 68.2%
Minibatch loss at step 150: 0.419523
Minibatch accuracy: 93.8%
Validation accuracy: 79.0%
Minibatch loss at step 200: 0.578647
Minibatch accuracy: 68.8%
Validation accuracy: 77.9%
Minibatch loss at step 250: 1.243334
Minibatch accuracy: 62.5%
Validation accuracy: 79.3%
Minibatch loss at step 300: 0.693099
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 350: 0.703617
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 400: 0.364670
Minibatch accuracy: 93.8%
Validation accuracy: 81.9%
Minibatch loss at step 450: 0.302830
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 500: 0.578489
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [57]:
from math import ceil, pow

batch_size = 16
num_hidden = 84
patch_size = {'conv1':5, 'pool1':2, 'conv2':5, 'pool2':2, 'conv3':4}
depth = {'conv1':6, 'conv2':16, 'conv3':120}
stride = {'conv1':1, 'pool1':2, 'conv2':1, 'pool2':2, 'conv3':1}
padding = {'conv1':'VALID', 'pool1':'SAME', 'conv2':'VALID', 'pool2':'SAME', 'conv3':'VALID'}
dropout_keep_prob = 0.5
starter_learning_rate = 0.1

def calc_output_size(in_size, patch, stride, padding):
    if padding == 'VALID':
        return int(ceil((in_size - patch + 1) / stride))
    else:
        return int(ceil(in_size / stride))
    
def calc_output_size_after_conv(in_size, patch, stride, padding):
    layer1_conv_HW = calc_output_size(in_size, patch['conv1'], stride['conv1'], padding['conv1'])
    layer1_pool_HW = calc_output_size(layer1_conv_HW, patch['pool1'], stride['pool1'], padding['pool1'])
    layer2_conv_HW = calc_output_size(layer1_pool_HW, patch['conv2'], stride['conv2'], padding['conv2'])
    layer2_pool_HW = calc_output_size(layer2_conv_HW, patch['pool2'], stride['pool2'], padding['pool2'])
    layer3_conv_HW = calc_output_size(layer2_pool_HW, patch['conv3'], stride['conv3'], padding['conv3'])
    return int(layer3_conv_HW)

def init_weight_stddev(fan_in_size):
    return np.sqrt(2.0/float(fan_in_size))

HW_after_conv = int(pow(calc_output_size_after_conv(image_size,patch_size,stride,padding),2) * depth['conv3'])

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = {'conv1': tf.Variable(tf.truncated_normal(
      [patch_size['conv1'], patch_size['conv1'], num_channels, depth['conv1']], stddev=init_weight_stddev(patch_size['conv1']*patch_size['conv1']*num_channels))),
            'conv2': tf.Variable(tf.truncated_normal(
      [patch_size['conv2'], patch_size['conv2'], depth['conv1'], depth['conv2']], stddev=init_weight_stddev(patch_size['conv2']*patch_size['conv2']*depth['conv1']))),
            'conv3': tf.Variable(tf.truncated_normal(
      [patch_size['conv3'], patch_size['conv3'], depth['conv2'], depth['conv3']], stddev=init_weight_stddev(patch_size['conv3']*patch_size['conv3']*depth['conv2']))),
            'h1': tf.Variable(tf.truncated_normal(
      [HW_after_conv, num_hidden], stddev=init_weight_stddev(HW_after_conv))),
            'h2': tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=init_weight_stddev(num_hidden)))}
  biases = {'conv1': tf.Variable(tf.zeros([depth['conv1']])),
           'conv2': tf.Variable(tf.zeros([depth['conv2']])),
           'conv3': tf.Variable(tf.zeros([depth['conv3']])),
           'h1': tf.Variable(tf.zeros([num_hidden])),
           'h2': tf.Variable(tf.zeros([num_labels]))}

  # Model.
  def model(data, keep_prob = 1):
    conv = tf.nn.conv2d(data, weights['conv1'], [1, stride['conv1'], stride['conv1'], 1], padding=padding['conv1'])
    relu = tf.nn.relu(conv + biases['conv1'])
    hidden = tf.nn.max_pool(relu, [1, patch_size['pool1'], patch_size['pool1'], 1], [1, stride['pool1'], stride['pool1'], 1], padding=padding['pool1'])
    conv = tf.nn.conv2d(hidden, weights['conv2'], [1, stride['conv2'], stride['conv2'], 1], padding=padding['conv2'])
    relu = tf.nn.relu(conv + biases['conv2'])
    hidden = tf.nn.max_pool(relu, [1, patch_size['pool2'], patch_size['pool2'], 1], [1, stride['pool2'], stride['pool2'], 1], padding=padding['pool2'])
    conv = tf.nn.conv2d(hidden, weights['conv3'], [1, stride['conv3'], stride['conv3'], 1], padding=padding['conv3'])
    hidden = tf.nn.relu(conv + biases['conv3'])
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    #dropout = tf.nn.dropout(reshape, keep_prob)
    relu = tf.nn.relu(tf.matmul(reshape, weights['h1']) + biases['h1'])
    hidden = tf.nn.dropout(relu, keep_prob)
    return tf.matmul(hidden, weights['h2']) + biases['h2']
  
  global_step = tf.Variable(0, trainable=False)

  # Training computation.
  logits = model(tf_train_dataset, dropout_keep_prob)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [58]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.192449
Minibatch accuracy: 18.8%
Validation accuracy: 12.6%
Minibatch loss at step 50: 1.633941
Minibatch accuracy: 43.8%
Validation accuracy: 44.2%
Minibatch loss at step 100: 1.283451
Minibatch accuracy: 50.0%
Validation accuracy: 63.5%
Minibatch loss at step 150: 0.887120
Minibatch accuracy: 68.8%
Validation accuracy: 73.6%
Minibatch loss at step 200: 0.657307
Minibatch accuracy: 68.8%
Validation accuracy: 74.3%
Minibatch loss at step 250: 1.479261
Minibatch accuracy: 50.0%
Validation accuracy: 76.7%
Minibatch loss at step 300: 0.569173
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 350: 0.658400
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.511610
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.336585
Minibatch accuracy: 100.0%
Validation accuracy: 81.6%
Minibatch loss at step 500: 1.043257
Minibatch accuracy: 68.8%
Validation accuracy: 82.0%
